---

## Floating-point arithmetic

Real numbers are stored on a computer following the IEEE floating-point standard:

1. **half precision** using 16 bits (Julia type: `Float16`)
2. **single precision** using 32 bits (Julia type: `Float32`)
3. **double precision** using 64 bits (Julia type: `Float64`)

Julia also has an **arbitrary precision** floating-point data type called `BigFloat`. It is excellent if you need more precision, but it is also much slower.

In [1]:
?AbstractFloat

search: AbstractFloat AbstractArray AbstractDict AbstractSet AbstractChar



```
AbstractFloat <: Real
```

Abstract supertype for all floating point numbers.


In [2]:
subtypes(AbstractFloat)

5-element Vector{Any}:
 BigFloat
 Core.BFloat16
 Float16
 Float32
 Float64

In [3]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

In [4]:
supertype(Real)

Number

In [5]:
subtypes(Number)

3-element Vector{Any}:
 Base.MultiplicativeInverses.MultiplicativeInverse
 Complex
 Real

In [6]:
im^2

-1 + 0im

In [7]:
supertype(Number)

Any

In [8]:
subtypes(Any)

649-element Vector{Any}:
 AbstractArray
 AbstractChannel
 AbstractChar
 AbstractDict
 AbstractDisplay
 AbstractMatch
 AbstractPattern
 AbstractSet
 AbstractString
 Any
 Base.ANSIDelimiter
 Base.ANSIIterator
 Base.AbstractBroadcasted
 ⋮
 Tuple
 Type
 TypeVar
 UndefInitializer
 Val
 VecElement
 VersionNumber
 WeakRef
 ZMQ.Context
 ZMQ.Socket
 ZMQ.lib.zmq_msg_t
 ZMQ.lib.zmq_pollitem_t

---

## Description of IEEE double floating-point format (`Float64`)

Suppose $x$ is a floating-point number stored in the following 64-bits:

$$
\begin{array}{|c|c|c|c|c|c|c|}
\hline
1 & 2 & \cdots & 12 & 13 & \cdots & 64 \\
\hline
s & e_{10} & \cdots & e_0 & f_1 & \cdots & f_{52} \\
\hline
\end{array}
$$

- 1 bit $s$ represents the **sign**
- 11 bits $e_{10} \cdots e_{0}$ represent the **exponent**
- 52 bits $f_1 \cdots f_{52}$ represent the **fraction** (a.k.a. the mantissa or significand)

Then

$$ x = (-1)^s [1.f_1 \cdots f_{52}]_2 \times 2^{(e-1023)}.$$

Notes: 

- $x$ is **normalized** to have its first digit nonzero.
- $e = [e_{10} \cdots e_{0}]_2 = e_{10} 2^{10} + \cdots + e_1 2^1 + e_0 2^0 \in \left[0, 2^{11}-1\right] = [0, 2047]$
- $e = 0$ and $e = 2047$ are reserved for special floating-point values, so 

$$e \in [1, 2046]$$

- the "$-1023$" in the exponent is called the **bias**:  $e-1023 \in [-1022,1023]$
- $[1.f_1 \cdots f_{52}]_2 = 1 + \frac{f_1}{2^1} + \frac{f_2}{2^2} + \cdots + \frac{f_{52}}{2^{52}}$


---

## Example

$$
\begin{split}
x & = -[1.101101]_2 \times 2^{(1026-1023)} \\
  & = -[1.101101]_2 \times 2^{3} \\
  & = -[1101.101]_2 \\
  & = -\left(1 \cdot 8 + 1 \cdot 4 + 0 \cdot 2 + 1 \cdot 1 + 1 \cdot \frac{1}{2} + 0 \cdot \frac{1}{4} + 1 \cdot \frac{1}{8}\right)  \\
  & = -13.625
\end{split}
$$

In [3]:
?bitstring

search: bitstring Cstring string Cwstring SubString isprint Missing String



```
bitstring(n)
```

A string giving the literal bit representation of a primitive type.

See also [`count_ones`](@ref), [`count_zeros`](@ref), [`digits`](@ref).

# Examples

```jldoctest
julia> bitstring(Int32(4))
"00000000000000000000000000000100"

julia> bitstring(2.2)
"0100000000000001100110011001100110011001100110011001100110011010"
```


In [9]:
s = bitstring(-13.625)

"1100000000101011010000000000000000000000000000000000000000000000"

In [10]:
function mybitstring(x)
    s = bitstring(x)
    s[1], s[2:12], s[13:64]
end

mybitstring (generic function with 1 method)

In [12]:
mybitstring(-13.625)

('1', "10000000010", "1011010000000000000000000000000000000000000000000000")

In [13]:
0b10000000010

0x0402

In [14]:
Int(0b10000000010)

1026

---

## Example

Even if a number can be represented exactly in base-10 with a finite number of digits, it may require an infinite number of digits in base-2.

$$
0.1 = \left[0.000110011001\ldots\right]_2 = \left[1.\overline{1001}\right]_2 \times 2^{-4}
$$

Therefore, $0.1$ cannot be represented exactly as a floating-point number.

In [15]:
mybitstring(0.1)

('0', "01111111011", "1001100110011001100110011001100110011001100110011010")

In [16]:
BigFloat(0.1)

0.1000000000000000055511151231257827021181583404541015625

In [17]:
BigFloat("0.1")

0.1000000000000000000000000000000000000000000000000000000000000000000000000000002

In [18]:
10.0*0.1 == 1.0

true

In [19]:
1//10

1//10

---

## Limits of floating-point numbers

- **Largest** `Float64` $= \left(2 - 2^{-52}\right) \times 2^{1023} \approx 2 \times 10^{308}$
- **Smallest positive normalized** `Float64` $= 2^{-1022} \approx 2 \times 10^{-308}$

In [20]:
?floatmax

search: floatmax floatmin float findmax Cfloat Float32 Float16 Float64



```
floatmax(T = Float64)
```

Return the largest finite number representable by the floating-point type `T`.

See also: [`typemax`](@ref), [`floatmin`](@ref), [`eps`](@ref).

# Examples

```jldoctest
julia> floatmax(Float16)
Float16(6.55e4)

julia> floatmax(Float32)
3.4028235f38

julia> floatmax()
1.7976931348623157e308

julia> typemax(Float64)
Inf
```


In [21]:
# Experiment with floatmax
x = floatmax(Float64)

1.7976931348623157e308

In [22]:
mybitstring(x)

('0', "11111111110", "1111111111111111111111111111111111111111111111111111")

In [24]:
(x + 1.0) - x

0.0

In [25]:
x + x # 2x

Inf

In [26]:
1.0/Inf

0.0

In [27]:
Inf - Inf

NaN

In [28]:
NaN + 1.0

NaN

---

In [29]:
?floatmin

search: floatmin floatmax float Cfloat findmin Float32 Float16 Float64



```
floatmin(T = Float64)
```

Return the smallest positive normal number representable by the floating-point type `T`.

# Examples

```jldoctest
julia> floatmin(Float16)
Float16(6.104e-5)

julia> floatmin(Float32)
1.1754944f-38

julia> floatmin()
2.2250738585072014e-308
```


In [30]:
# Experiment with floatmin
x = floatmin(Float64)

2.2250738585072014e-308

In [31]:
mybitstring(x)

('0', "00000000001", "0000000000000000000000000000000000000000000000000000")

In [32]:
x*floatmax()

3.9999999999999996

In [33]:
1/x

4.49423283715579e307

In [34]:
x/2.0

1.1125369292536007e-308

In [35]:
mybitstring(x/2.0)

('0', "00000000000", "1000000000000000000000000000000000000000000000000000")

In [36]:
x/floatmax()

0.0

---

## De-normalized floating-point numbers

The IEEE floating-point standard also allows **de-normalized** numbers that are smaller than `floatmin(Float64)`. De-normalized floats are represented by $e = 0$.

In [43]:
# Compute the smallest Float64 that is not zero
#('0', "00000000000", "0000000000000000000000000000000000000000000000000001")
# x = [0.0000000000000000000000000000000000000000000000000001]_2 * 2^(-1022)
#   = ( 0*2^(-1) + ... + 0*2^(-51) + 1*2^(-52) ) * 2^(-1022)
#   = 2^(-1074)
x = 2.0^(-1074)
mybitstring(x)

('0', "00000000000", "0000000000000000000000000000000000000000000000000001")

In [44]:
x

5.0e-324

In [45]:
x/2.0

0.0

In [46]:
x > 0

true

In [47]:
1.0/(x/2.0)

Inf

In [48]:
2.0/x

Inf

---

## Other special floats

- `0.0` and `-0.0`: $$e_{10} \cdots e_0 = 0 \cdots 0 \quad \text{and} \quad f_1 \cdots f_{52} = 0 \cdots 0$$
- `Inf` and `-Inf`: $$e_{10} \cdots e_0 = 1 \cdots 1 \quad \text{and} \quad f_1 \cdots f_{52} = 0 \cdots 0$$
- `NaN` (not-a-number): $$e_{10} \cdots e_0 = 1 \cdots 1 \quad \text{and} \quad f_1 \cdots f_{52} \neq 0$$

From [Julia Manual: Mathematical Operations and Elementary Functions](https://docs.julialang.org/en/v1/manual/mathematical-operations/):

- Finite numbers are ordered in the usual manner.
- Positive zero is equal but not greater than negative zero.
- `Inf` is equal to itself and greater than everything else except `NaN`.
- `-Inf` is equal to itself and less then everything else except `NaN`.
- `NaN` is not equal to, not less than, and not greater than anything, including itself.

In [ ]:
mybitstring(NaN)

In [49]:
# Experiment with 0.0, -0.0, Inf, -Inf, and NaN
0.0 == -0.0

true

In [50]:
0.0 > -0.0

false

In [51]:
Inf == Inf

true

In [53]:
Inf > floatmax()

true

In [54]:
Inf > Inf

false

In [55]:
Inf > NaN

false

In [56]:
NaN == NaN

false

In [57]:
?===

search: === == !==



```
===(x,y) -> Bool
≡(x,y) -> Bool
```

Determine whether `x` and `y` are identical, in the sense that no program could distinguish them. First the types of `x` and `y` are compared. If those are identical, mutable objects are compared by address in memory and immutable objects (such as numbers) are compared by contents at the bit level. This function is sometimes called "egal". It always returns a `Bool` value.

# Examples

```jldoctest
julia> a = [1, 2]; b = [1, 2];

julia> a == b
true

julia> a === b
false

julia> a === a
true
```


In [58]:
NaN === NaN

true

In [59]:
mybitstring(0.0)

('0', "00000000000", "0000000000000000000000000000000000000000000000000000")

In [60]:
mybitstring(-0.0)

('1', "00000000000", "0000000000000000000000000000000000000000000000000000")

In [61]:
1.0/0.0

Inf

In [62]:
1.0/-0.0

-Inf

In [63]:
0.0 > -0.0

false

In [65]:
x = 2.0^(-1074)

5.0e-324

In [67]:
x/(-2.0)

-0.0

---

## Machine epsilon `eps(Float64)` and the unit roundoff $\eta$

- `1.0 + eps(Float64)` is the first `Float64` that is larger than `1.0`

$$\mathtt{eps(Float64)} = 2^{-52} \approx 2.2 \times 10^{-16}$$

- $\eta = $ `eps(Float64)/2.0` is the largest possible **relative error** due to roundoff

$$\eta = 2^{-53} \approx 1.1 \times 10^{-16}$$

In [68]:
-floatmax(),floatmax()

(-1.7976931348623157e308, 1.7976931348623157e308)

In [69]:
?eps

search: eps exp2 keys less else repr



```
eps(::Type{T}) where T<:AbstractFloat
eps()
```

Return the *machine epsilon* of the floating point type `T` (`T = Float64` by default). This is defined as the gap between 1 and the next largest value representable by `typeof(one(T))`, and is equivalent to `eps(one(T))`.  (Since `eps(T)` is a bound on the *relative error* of `T`, it is a "dimensionless" quantity like [`one`](@ref).)

# Examples

```jldoctest
julia> eps()
2.220446049250313e-16

julia> eps(Float32)
1.1920929f-7

julia> 1.0 + eps()
1.0000000000000002

julia> 1.0 + eps()/2
1.0
```

---

```
eps(x::AbstractFloat)
```

Return the *unit in last place* (ulp) of `x`. This is the distance between consecutive representable floating point values at `x`. In most cases, if the distance on either side of `x` is different, then the larger of the two is taken, that is

```
eps(x) == max(x-prevfloat(x), nextfloat(x)-x)
```

The exceptions to this rule are the smallest and largest finite values (e.g. `nextfloat(-Inf)` and `prevfloat(Inf)` for [`Float64`](@ref)), which round to the smaller of the values.

The rationale for this behavior is that `eps` bounds the floating point rounding error. Under the default `RoundNearest` rounding mode, if $y$ is a real number and $x$ is the nearest floating point number to $y$, then

$$
|y-x| \leq \operatorname{eps}(x)/2.
$$

See also: [`nextfloat`](@ref), [`issubnormal`](@ref), [`floatmax`](@ref).

# Examples

```jldoctest
julia> eps(1.0)
2.220446049250313e-16

julia> eps(prevfloat(2.0))
2.220446049250313e-16

julia> eps(2.0)
4.440892098500626e-16

julia> x = prevfloat(Inf)      # largest finite Float64
1.7976931348623157e308

julia> x + eps(x)/2            # rounds up
Inf

julia> x + prevfloat(eps(x)/2) # rounds down
1.7976931348623157e308
```

---

```
eps(::Type{DateTime}) -> Millisecond
eps(::Type{Date}) -> Day
eps(::Type{Time}) -> Nanosecond
eps(::TimeType) -> Period
```

Return the smallest unit value supported by the `TimeType`.

# Examples

```jldoctest
julia> eps(DateTime)
1 millisecond

julia> eps(Date)
1 day

julia> eps(Time)
1 nanosecond
```


In [70]:
?nextfloat

search: nextfloat maxintfloat nextprod nextpow float Cfloat prevfloat



```
nextfloat(x::AbstractFloat, n::Integer)
```

The result of `n` iterative applications of `nextfloat` to `x` if `n >= 0`, or `-n` applications of [`prevfloat`](@ref) if `n < 0`.

---

```
nextfloat(x::AbstractFloat)
```

Return the smallest floating point number `y` of the same type as `x` such `x < y`. If no such `y` exists (e.g. if `x` is `Inf` or `NaN`), then return `x`.

See also: [`prevfloat`](@ref), [`eps`](@ref), [`issubnormal`](@ref).


In [71]:
?prevfloat

search: prevfloat prevpow float Cfloat nextfloat



```
prevfloat(x::AbstractFloat, n::Integer)
```

The result of `n` iterative applications of `prevfloat` to `x` if `n >= 0`, or `-n` applications of [`nextfloat`](@ref) if `n < 0`.

---

```
prevfloat(x::AbstractFloat)
```

Return the largest floating point number `y` of the same type as `x` such `y < x`. If no such `y` exists (e.g. if `x` is `-Inf` or `NaN`), then return `x`.


In [72]:
# Experiment with eps
ϵ = eps()

2.220446049250313e-16

In [74]:
1.0 + eps()

1.0000000000000002

In [75]:
nextfloat(1.0)

1.0000000000000002

In [76]:
(1.0 + eps()) - 1.0

2.220446049250313e-16

In [73]:
# Unit roundoff
η = ϵ/2.0

1.1102230246251565e-16

In [80]:
(1.0 + η)

1.0

In [78]:
(1.0 + η) - 1.0

0.0

In [81]:
# The gap between floats in [1,2] is 2^-52
x = 1.5
nextfloat(x) - x

2.220446049250313e-16

In [82]:
# The gap between floats in [2,4] is 2^-51
x = 2.0
nextfloat(x) - x

4.440892098500626e-16

In [83]:
# The gap between floats in [4,8] is 2^-50
x = 4.0
nextfloat(x) - x

8.881784197001252e-16

In [84]:
# Each interval [2^k, 2^(k+1)) has 1/2^-52 = 2^52 floats
2^52

4503599627370496

In [89]:
4_503_599_627_370_496  # approx. 4.5 trillion!

4503599627370496

In [85]:
# The gap between floats in [2^k, 2^(k+1)] is 2^(-52 + k)
# The gap between floats in [2^52, 2^53] is 2^0 = 1
x = 2.0^52
nextfloat(x) - x

1.0

In [86]:
# The gap between floats in [2^53, 2^54] is 2^1 = 2
# This is the smallest positive integer not representable in Float64
2^53+1

9007199254740993

In [87]:
9007199254740993.0

9.007199254740992e15

---

## Roundoff error example

Suppose we are using a base-10 floating-point system with 4 significant digits, using `RoundNearest`:

$$
\begin{split}
\left( 1.112 \times 10^1 \right) \times \left( 1.112 \times 10^2 \right)
& = 1.236544 \times 10^3 \\
& \rightarrow 1.237 \times 10^3 = 1237
\end{split}
$$

The absolute error is $1237 - 1236.544 = 0.456$.

The relative error is $$\frac{0.456}{1236.544} \approx 0.0004 = 0.04 \%$$

The default rounding mode is `RoundNearest` (round to the nearest floating-point number). This implies that

$$ \frac{|x - \mathrm{fl}(x)|}{|x|} \leq \eta.$$

If `RoundToZero` is used (a.k.a. **chopping**), then

$$ \frac{|x - \mathrm{fl}(x)|}{|x|} \leq 2 \eta.$$

`RoundNearest` is used since it produces smaller roundoff errors.

---

## Roundoff error accumulation

When performing arithmetic operations on floats, extra **guard digits** are used to ensure **exact rounding**. This guarantees that the relative error of a floating-point operation (**flop**) is small. More precisely, for floating-point numbers $x$ and $y$, we have

$$
\begin{split}
\mathrm{fl}(x \pm y) &= (x \pm y)(1 + \varepsilon_1) \\
\mathrm{fl}(x \times y) &= (x \times y)(1 + \varepsilon_2) \\
\mathrm{fl}(x \div y) &= (x \div y)(1 + \varepsilon_3) \\
\end{split}
$$

where $|\varepsilon_i| \leq \eta$, for $i = 1,2,3$, where $\eta$ is the unit roundoff.

Although the relative error of each flop is small, it is possible to have the roundoff error accumulate and create significant error in the final result. If $E_n$ is the error after $n$ flops, then:

- **linear roundoff error accumulation** is when $E_n \approx c_0 n E_0$
- **exponential roundoff error accumulation** is when $E_n \approx c_1^n E_0$, for some $c_1 > 1$

In general, linear roundoff error accumulation is unavoidable. On the other hand, exponential roundoff error accumulation is not acceptable and is an indication of an **unstable algorithm**. (See Example 1.6 in Ascher-Greif for an example of exponential roundoff error accumulation, and see Exercise 5 in Section 1.4 for a numerically stable method to accomplish the same task.)

---

## General advice

1. Adding $x + y$ when $|x| \gg |y|$ can cause the information in $y$ to be 'lost' in the summation.

2. Dividing by very small numbers or multiplying by very large numbers can **magnify error**.

3. Subtracting numbers that are almost equal produces **cancellation error**.

4. An **overflow** occurs when the result is too large in magnitude to be representable as a float. Result will become either `Inf` or `-Inf`. Overflows should be avoided.

4. An **underflow** occurs when the result is too small in magnitude to be representable as a float. Result will become either `0.0` or `-0.0`.


---

## Example (summation order)

This next example shows that summation order can make a difference. We will compute

$$
s = \sum_{n = 1}^{1,000,000} \frac{1}{n}
$$

in two different ways: from largest to smallest and from smallest to largest.

---

## Example (cancellation error)

Show that 

$$
\ln\left( x - \sqrt{x^2-1} \right) = -\ln\left( x + \sqrt{x^2-1} \right).
$$

Which formula is more suitable for numerical computation?

In [ ]:
# Experiment with both formulas

x = 1e6
fl = log(x - sqrt(x^2 - 1))
fr = -log(x + sqrt(x^2 - 1))
fl, fr

---

## Example (avoiding overflow)

Overflow is possible when squaring a large number. This needs to be avoided when computing the Euclidean norm (a.k.a. the $2$-norm) of a vector $x$:

$$
\|x\|_2 = \sqrt{x_1^2 + x_2^2 + \cdots + x_n^2}.
$$

If some $x_i$ is very large, it is possible that $x_i^2$ will overflow, causing the final result to be `Inf`. We can avoid this as follows.

Let 
$$\bar{x} = \max_{i=1:n} |x_i|.$$
Then
$$
\|x\|_2 = \bar{x} \sqrt{\left(\frac{x_1}{\bar{x}}\right)^2 + \left(\frac{x_2}{\bar{x}}\right)^2 + \cdots + \left(\frac{x_n}{\bar{x}}\right)^2}.
$$
Since $|x_i/\bar{x}| \leq 1$ for all $i$, no overflow will occur. Underflow may occur, but this is harmless.


In [ ]:
# Experiment with both formulas
n = 100

x = rand(n)
x[1] = 1e200*rand()

using LinearAlgebra
norm(x)

In [ ]:
x[1]^2

In [ ]:
normx = 0.0
for i = 1:n
    normx += x[i]^2
end
normx = sqrt(normx)

---